# Tests

Tests are separate bits of code in addition to the main application code that are written to confirm and verify the desired behavior of various scopes of application. These are the responsibility of the `developer` and are typically written as the main application is being written. Each of these tests are **self contained**, in that each test is responsible for setting up the system before running the actual test and tearing it down after the test is run. 


## Test Types
Application tests can be grouped into three main categories, that differ in terms of the scope of the application code that they are written to verify (cover). These are

 1. Unit Tests
 1. Integration Tests
 1. End-to-End (Feature) Tests
 
### Unit Tests

 - A Unit test has the smallest scope or code coverate of testing where each unit tests checks a small `unit` of code in isolation. 
 - If the unit has any interaction with another dependent unit or module that dependency is `mocked` out.
 - Is the most fine-grained of all tests, requiring a lot of unit tests to cover the entire codebase.
 - Each test must execute as quickly as possible in order to lend itself to automation.
 
### Integration Tests
 
 - Tests that two or more units or modules work together (integrate) as expected.
  - eg. Test that checks for the interaction between server code and a database.
 - Can take longer to run than unit-tests due to remote calls and other integrations that need to run as part of the test
 - Requires more setup than unit tests (like setting up a fake server or test database to integrate with)
 
### End-to-End Tests

 - Also called `Feature Tests`, they test out a high level feature of the application
  - Tests the application at the highest level of abstraction, mostly oblivious to the way the app is implemented
 - These involve testing an application at the browser level, simulating real user interaction with the system
 - Take the longest to run, most expensive to write and most prone to failure
  - Involves communication between the client (browser) and server layers which will take the same time as a real feature execution
  - Run using tools like Selenium that has its own learning curve
  - Most prone to failure when changes are made to the system, increasing maintenance cost
  - Less deterministic than unit or integration tests
 
 
### Testing Pyramid
The rule of thumb for a typical application is to have the largest number of fine-grained unit tests, a smaller number of integration tests and then an even smaller set of end to end tests.

![Testing Pyramid](https://2.bp.blogspot.com/-YTzv_O4TnkA/VTgexlumP1I/AAAAAAAAAJ8/57-rnwyvP6g/s1600/image02.png "Testing Pyramid")

## Tests in Python

Testing support in python is mainly provided by the `unittest` module. This provides support for test automation, sharing of test setup and teardown across individual test cases, Grouping of related tests as `Suites` of tests and finally, a report generation framework to print out the results of the test run.

`unittest` as the name suggests, is mainly used to write **unit tests**, however they can be used to write **integration tests** as well.

### Basic Components

In order to write tests for the `unittest` module to run, you would need to provide the following main components in order to be compliant with unittest's automation test runner.

#### TestCase
A class that encompasses a set of related tests. This class would need to be a `subclass` of the module provided `unittest.TestCase` class.

```python
import unittest
class MyTests(unittest.TestCase):

```

#### Individual Tests
Each individual test is written as an instance function in the TestCase class. In order for these methods to be triggered by the test runner, their names must be of the form **test_***

```python
class MyTests(unittest.TestCase):
    def test_myunit_test(self):
        # assertions
        
```

#### Assertion statements
The assertion statements are found within each test and is how the expected behavior of the functionality under test is checked. The `unittest.TestCase` class provides a large set of instance `assert*` functions that you can call to make various types of assertions.

```python
class MyTestCase(unittest.TestCase):
    def test_myunit_test(self):
        self.assertEqual(1, 1, "Expected the output to be 1")
```

A list of the more commonly used assert statements can be found [here](https://docs.python.org/3.5/library/unittest.html#assert-methods)

#### setUp and tearDown
Helper methods of the `unittest.TestCase` class that can be overridden to provide setUp and teardown for each test. The setup method is run before the execution of each test and the teardown is run immediately after each test is run. The setup method can also be used to create any kind of shared data used by the tests. This ensures that the TestCase class is `DRY` so that code that is commonly required by all `test_*` methods is written once and re-used.

These methods are what makes the `TestCase` class self-contained. Each run of the `TestCase` class must manage its own setup required for the tests to run, and once completed must also be responsible for any clean-up operations required to leave the system in a good state.

```python
class MyTestCase(unittest.TestCase):
    def setUp(self):
        self.dep_system = DepSystem()
        self.shared_data = list([0, 1, 2, 3])
        
    def tearDown(self):
        self.dep_system.shutDown()
```

#### TestSuite
`unittest.TestSuite` is a class provided to create groups or `suites` of TestCases. 

```python
def suite():
    suite = unittest.TestSuite()
    suite.addTest(MyTestCase('test_myunit_test'))
    suite.addTest(MyOtherTestCase('test_myother_test'))
    return suite
```

The `unittest` Test runner provides default grouping for tests in each TestCase class for you so unless you want custom grouping into suites, this is less frequently used.

#### Decorators
There are several test method decorators that you can use to determine the set of tests that you want to run

 1. **@unittest.skip** 
Decorating a `test_*` method with `@unittest.skip` will cause the test runner to skip this test. This test will be marked as skipped in the test run report

 1. **@unittest.skipif**
Conditionally skip a test

 1. **@unittest.expectedFailure**
If you know a test is set to fail, you can use this decorator to prevent it messing up your test report

### Example

Consider the following example class that tests out the functionality of the String operations

In [1]:
import unittest
import datetime

class StringTests(unittest.TestCase):
    """Unit testing class for Strings"""

    def setUp(self):
        self.str = "TestString"

    def test_concat_strings(self):
        stra = "Word"

        self.assertEqual(self.str + stra, "TestStringWord", "It should concatenate strings")

    def test_lower(self):
        self.assertEqual(self.str.lower(), "teststring", "It should convert all characters to lower case")
        
    def test_format(self):
        result = "Format check {}".format(2)
        self.assertEqual(result, "Format check 2", "formatting not done correctly")
        
    def test_isdigit(self):
        with self.assertRaises(TypeError):
            self.str.split(2)
        
    @unittest.skipIf(datetime.date.today().month > 2, "Can't run this test after Feb")    
    def test_length(self):
        result = len(self.str)
        self.assertEqual(result, 10, "String is of length 10")

    def tearDown(self):
        pass
        
if __name__ == "__main__":
    # Arguments to main added due to iPython. You'd run this as unittest.main()
    unittest.main(argv=["Ignored"], exit=False)

...s.
----------------------------------------------------------------------
Ran 5 tests in 0.004s

OK (skipped=1)


*TestCases must be **entirely self-contained units of code** that handle all of the setup and teardown that the individual test assertions require. They should **NOT** rely on any kind of manual intervention. This makes them scriptable and "automatable" (TestCases will typically be run as part of an automated build script).*

Additionally, the tests should produce minimal output about its inner details. The only output from the tests should be whether the tests all passed and if not which ones failed and why. For failures, it may provide stacktraces to provide points of further investigation.

### Exercise

Write a simple class called `ArithmeticCalculator` that exposes functions that perform the simple arithmetic operations of `add`, `subtract`, `multiply` and `divide`. Then write a `TestCase` that contains unit tests that test out the functionality of the exposed functions.

#### Directory Structure
All tests should be written inside a `tests` folder that will be in your `src` folder. The directory structure of the tests folder should mirror that of the src folder.

#### Execution Instructions
You should unittests Test runner to execute your tests

 1. Using the unittest test runner on the command line
 This will run all of the tests that are in the file `<filename>.py`.
 ```bash
 python -m unittest tests/<filename>.py
 ```
 
 1. Using the discover mechanism
 This will look for all files under the tests directory with the filename that matches the pattern `*.py` and run them
 ```bash
 python -m unittest discover -s tests -p *.py
 ```
 
 1. Triggering the test runner from within your code
 ```python
 if __name__ == "__main__":
     unittest.main()
 ```
 
There are several command line options that you can provide to the unittest Test Runner which are described [here](https://docs.python.org/3.5/library/unittest.html#command-line-options)

## Test Doubles

When writing tests, it is important to ensure the scope of your tests are localized. Especially for unit tests, your tests must focus exclusively on the unit of code under test, with no reliance on any of that unit's dependencies. That's where `test doubles` come in.

A **`Test Double`** is code (typically a class or a function) that simplifies and simulates working production code that is a dependency of the unit of code under test. *The term comes from the movie industry where you have `stunt doubles` the replace real actors in dangerous scenes*.

eg. Assume the unit under test is a class that makes a call to a database to retrieve data, run some calculations on it and generate a report. In this case, your unit tests should be written to confirm the logic of the calculations and report generation and **NOT** be making actual calls to the database. Database calls will require setup and teardown of actual data which will also cause your tests to take longer to execute. More importantly, you will in all likelihood use a thoroughly tested, in-built library to make the network call to the db, therefore there is no need for your unit tests to test it out as well. Instead, you should make use of a `stub` object that simulates the behavior of a database by returning a fixed set of rows depending on the calculations you want to test. This will make your unit tests much more predictable and quicker to complete.

### Types
There are five different types of `Test Doubles` that are used in the industry.

 1. **Dummy** These are instance objects that never actually get used. 
  1. They're typically used to fill un-interseting required arguments in a function call
 1. **Stub** These are either objects or functions that return a fixed value. 
  1. Used to confirm behavior of your unit under test when its dependency returns given values.
 1. **Spy** A Spy is a Stub, with the added logic to record calls made to it.
  1. Used when you want to confirm that a dependency is called `n` number of times and with a certain set of parameters.
 1. **Mock** A mock is a Spy setup with pre-defined expectations of interactions and also includes verification logic that checks that expectations are met.
  1. An example mock object could be setup to be called `2` times with `x` and `y` as its arguments. When the test is complete you would call is `verify` function to have it ensure that it was indeed called `2` times with the right arguments.
 1. **Fake** This is working code used to simulate a larger scope of production code
  1. Is used when the real system is not available or usable. eg. When a vendor writes code that uses a firm's proprietary software. Code at the vendor's site has no access to the software and so will make use of a `Fake` to simulate it and test against.
  1. Not uncommon for Fakes to have additional unit tests for itself.

## Test Doubles in Python
Support for test doubles in python is provided by the unittest module's `mock object library`. 

### unittest Mocks
The main classes are `Mock` and `MagicMock` that allow you to create different kinds of test doubles throughout your code and also provide helper assertion methods that let you monitor how they have been used (eg. how many times they've been called, with which arguments etc.)

*MagicMock* is a subclass of Mock with support for mocking python's `magic` methods (`__str__`)

In [26]:
import unittest
from unittest import TestCase
from unittest.mock import Mock, MagicMock, patch


class Calculation:
    crap = "Some crap"
    
    def run(self):
        return 10 * 10

    def debug_run(self):
        pass


class Calculator:
    def multiply(self, val, exponent):
        result = val

        for i in range(1, exponent):
            result *= val

        return result

    def square_and_increment(self, val):
        return self.multiply(val, 2) + 1

    def invoke(self, calculation):
        return calculation.run()

    def debug_invoke(self, calculation):
        return calculation.debug_run()


class MockTestCases(TestCase):
    def test_stub(self):
        calculator = Calculator()

        # Stub out a dependent method since the focus of this test is not multiply.
        calculator.multiply = Mock()
        calculator.multiply.return_value = 100

        result = calculator.square_and_increment(10)
        calculator.multiply.assert_called_once_with(10, 2)
        self.assertEqual(result, 101, "Square and increment of 10 should be 101")
        
    def test_mock_invocation(self):
        # Mocking out an object to ensure its member methods are called as expected
        calculation = MagicMock()
        calculator = Calculator()
        calculator.invoke(calculation)

        calculation.run.assert_called_with()

    def test_return_value(self):
        mock = Mock()
        mock.sample_field = 30
        mock.return_value = 10

        mock.sample_method.return_value = 20

        self.assertEqual(mock(), 10)
        self.assertEqual(mock.sample_field, 30)
        self.assertEqual(mock.sample_method(), 20)

    # Side effects.
    def test_side_effects(self):
        # Setup a stub to throw an exection
        mock = Mock()
        mock.debug_run.side_effect = Exception("Erroneous call")

        calculator = Calculator()
        
        with self.assertRaises(Exception):
            calculator.debug_invoke(mock)

    def test_stub_function(self):
        # Or have the mock call another canned function
        def canned_func():
            print("Call to the dummy stub function, do stub stuff here")
            return 500

        mock = Mock()
        mock.debug_run.side_effect = canned_func
        mock.crap.return_value = 100

        calculator = Calculator()
        self.assertEqual(calculator.debug_invoke(mock), 500)

    # Creating a mock based on a class definition
    def test_mock_spec(self):
        mock = MagicMock(spec=Calculation)
        mock.debug_run.return_value = 10
        
        # Spec'd mock should throw an error here because Calculation has no attribute called crap.
        mock.crap.return_value = 100

        calculator = Calculator()
        calculator.debug_invoke(mock)
    
    
if __name__ == "__main__":
    # Arguments to main added due to iPython. You'd run this as unittest.main()
    unittest.main(argv=["Ignored"], exit=False)

......

Call to the dummy stub function, do stub stuff here



----------------------------------------------------------------------
Ran 6 tests in 0.007s

OK


### Patch: Use of Mocks in Test Cases

The simplest way to create mocks is with the @patch decorator. The specified class or object will be replaced with instances of `MagicMock` and passed as arguments to the decorated function.

```python
import unitest.mock.patch

@patch("othermodule.OtherClass")
def test_some_method(self, otherClassMock):
        otherClassMock.other_function.return_value = 10
        
        result = otherClassMock.other_function()
        
        self.assertEqual(request, 10)
```

Remember, the use of mocks in a unit test case is to setup test doubles for the `dependencies` of the unit of code under test, so that your tests can focus on it's functionality rather than test out anything in the dependency. 

### Example

Consider the following class that exposes a function to delete a file. It first checks to see whether the file exists and if so delete it, and it uses the in-built os library to do so.

```python
import os
import os.path

class RemovalService(object):
    """Sample class that wraps the rm os call"""

    def rm(self, file_name):
        if os.path.isfile(file_name):
            os.remove(file_name)
```     


The Unit TestCase class to for the RemovalService class should have tests that confirm the logic decision of RemovalService, **NOT** the in-built os library. i.e. Your tests should **NOT** actually delete a file off the system. You accomplish this by providing mocks for the in-built `os` and `os.patch` modules.


```python
from mymodule import RemovalService

from unittest import TestCase
from unittest.mock import patch

class RemovalServiceTestCase(TestCase)
    @patch('mymodule.os.path')
    @patch('mymodule.os')
    def test_rm_on_a_nonexistent_file(self, mock_os, mock_path):
        """Testing the rm method on a non-existent file"""
        
        # Setup the mock to simulate a file that doesn't exist
        mock_path.isfile.return_value = False

        service = RemovalService()
        service.rm("my_file") # Doesn't need to be the path to an actual file since we're using mocks.

        self.assertFalse(mock_os.remove.called, "The remove call should not go through on a non-existent file")

    @patch('mymodule.os.path')
    @patch('mymodule.os')
    def test_rm_on_an_existing_file(self, mock_os, mock_path):
        """Testing the rm method for a file that does exist"""
        mock_path.isfile.return_value = True

        service = RemovalService()
        service.rm("my_file")

        # Check that the call went through with the right arguments
        mock_os.remove.assert_called_with("my_file")
```

Points to note

 * The mocks that you create using `@patch` should on the actual classes that Class under test imports. Note how the argument to patch is `mymodule.os.path` and not just `os.path`. You want the instance of `os` that got imported in `RemovalService`
 * Mocks inserted using nested `@patch` are done in reverse order. `mymodule.os` is the closest decorator to the `test_*` functions so its mock is the first parameter to those functions. `mymodule.os.patch` is the outer decorator so it goes as the second argument
 * The mocks are created on the **dependencies** of `RemovalService`. Since we want to test out `RemovalService` we mock out the other dependent classes of functions that it calls, in order to keep our test focus only on the `RemovalService` logic.
 * This TestCase is entirely self-contained. Through the use of mocks it does not depend on any file to exist or require any actual file to be deleted, and thus can be run on any machine with any setup.

## Test Driven Development (TDD)

Test Driven Development, relates to writing code as per specifications set by tests. The process involvles capturing user requirements, or system behaviour in small incremental tests, and then writing code to get those tests to pass. This typically results in building systems incrementally with small blocks of code and then building additional behavior that sits on top of previously built code, taking one test at a time and getting it to pass.

The process follows three main steps that are then repeated over and over again.

 - Red
  - Write a test before the code (causing it to fail). The missing code causes the test to go `Red`.
 - Green
  - Write the code to get the test to pass using the `Quickest path to Green` method.
 - Refactor
  - Refactor the code to better structure it whilst keeping the tests green. The Green tests ensure you are starting this test with working code.

Repeat with the next required behaviour.

### Advantages
 - Incremental build of the system allows its architecture to `evolve` as per the systems need.
  - As opposed to deciding an architecture up front which would most likely not capture all cases.
  - Results in a system that is more amenable to change.
 - Results in (close to) 100% test coverage, ensuring well tested code that inspires developer and user confidence
 - The tests ensure new features can be added or existing code can be modified without breaking functionality
 - Bugs or logic breaks are caught and fixed early in development process
  - Any code change that breaks existing logic will create red tests, leading to early bug detection and fixes
 - Results in the testing cycle being almost entirely automated with tests run dozens of times a day or on every build
  - *Important to keep tests small and their executions quick*
  
### Concerns
 - Can *appear* to take longer time due to `double` work
 - Counter-intuitive: Not natural to first timers to write a test before writing the real code
 - Can result in `brittle` tests: Any small change in the code result in the refactoring of the tests as well.
  - Usually occurs if the tests are written after the code
  - *This is why tests should check for `behaviour` not code*.
 - Different languages provide differing levels of support
 
In the real world BDD is implemented in various differing flavors. Teams often write the tests after writing code to have tests serve as safeguards against logic-breaking code modifications.

## Exercise

Use the concepts you have learned in this lesson to write unit tests for the option payoff calculation project. Tests much be written for each of the classes that check the logic of that class under test, with all of the dependencies of that class mocked out.

### Resources

[unittests in the Python standard library](https://docs.python.org/3.5/library/unittest.html)  
[unittest.mock](https://docs.python.org/3.5/library/unittest.mock.html)  
[unittest.mock examples](https://docs.python.org/3.5/library/unittest.mock-examples.html)